In [54]:
'''
Stellar Evolution Predictor
(Using Mass, Metallicity, and Monte Carlo Uncertainty)

This notebook predict's a star's final evolutionary remnant using astrophysical approximations:

Inputs (from user):
- initial mass in solar masses
- metallicity (Z)

Outputs:
- Main-sequence lifetime
- Effective final mass (M_eff) after mass loss
- Remnant classification
- Monte Carlo probability distribution of final outcomes
- Mass loss distribution (Bar chart)
- Nucleosynthetic yield visualization (Bar chart)

'''

"\nStellar Evolution Predictor\n(Using Mass, Metallicity, and Monte Carlo Uncertainty)\n\nThis notebook predict's a star's final evolutionary remnant using astrophysical approximations:\n\nInputs (from user):\n- initial mass in solar masses\n- metallicity (Z)\n\nOutputs:\n- Main-sequence lifetime\n- Effective final mass (M_eff) after mass loss\n- Remnant classification\n- Monte Carlo probability distribution of final outcomes\n- Mass loss distribution (Bar chart)\n- Nucleosynthetic yield visualization (Bar chart)\n\n"

In [55]:
import numpy as np, matplotlib.pyplot as plt
from collections import Counter
import re
import pandas as pd
from io import StringIO
import glob

In [56]:
#INPUT

In [57]:
#MAIN SEQUENCE LIFETIME

In [58]:
#MASS LOSS FUNCTION FROM METALLICITY Z

In [75]:
def parse_limongi(path):
    """
    Parse files with structure like:
    H Table: (M=1.0,Z=0.01)
    #H Lifetime: ...
    #H Mfinal: ...
    Isotopes Yields X0 Z A
    H-1 ...
    H-2 ...
    Returns: mass, dataframe with Isotopes + Yields
    """

    with open(path, "r") as f:
        lines = f.readlines()
        
    first_line = lines[0].strip()
    
    # locate M= and the next comma
    M_start = first_line.find("M=") + 2
    M_end = first_line.find(",", M_start)
    mass = float(first_line[M_start:M_end])

    #find table beginning
    table_start = None
    for i in range(1, len(lines)):
        line = lines[i].strip()
        if not line.startswith("#") and line != "":
            table_start = i
            break

    if table_start is None:
        raise ValueError("No table found in file.")


    df = pd.read_csv(
        path,
        delim_whitespace=True,
        comment="#",
        skiprows=table_start,
        names=["Isotopes", "Yields", "X0", "Z", "A"]
    )


    # -------------------------
    df["Yields"] = pd.to_numeric(df["Yields"], errors="coerce")
    df = df[["Isotopes", "Yields"]]

    return mass, df


In [76]:
parse_limongi("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z00002/M200_Z00002_copy.txt")

/var/folders/83/7k2j676j1wxfrsj948nvyhym0000gn/T/ipykernel_81423/2051150406.py:35: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


(2.0,
       Isotopes         Yields
 0     Isotopes            NaN
 1          H-1   2.449627e-01
 2          H-2   1.020761e-06
 3         He-3   7.895546e-05
 4         He-4   1.058293e-01
 5         Be-7   6.232244e-08
 6          B-8   7.082072e-15
 7         Li-7   7.563765e-11
 8         C-11   2.436165e-19
 9         B-11   2.111320e-11
 10        C-12   8.604548e-03
 11        C-13   8.087075e-04
 12        N-13   5.671939e-09
 13        N-14   5.303897e-04
 14        C-14   1.652648e-08
 15        N-15   5.143725e-08
 16        O-16   7.235706e-03
 17        O-17   4.901240e-06
 18        O-18   3.122700e-07
 19        F-17   9.696350e-13
 20        F-18   1.857641e-12
 21        F-19   1.442185e-06
 22       Ne-20   6.030262e-05
 23       Ne-21   3.012668e-07
 24       Ne-22   1.114584e-03
 25       Na-22   5.646416e-08
 26       Na-23   2.314616e-05
 27       Mg-23   2.205839e-18
 28       Mg-24   1.323722e-05
 29       Mg-25   2.660248e-06
 30       Mg-26   4.038405e-06
 3